Conecetando com o Google Drive:

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
# List the content of your local computer folder
!ls -la "/content/gdrive/My Drive/darknet"

In [ ]:
!sudo apt-get install tree

In [ ]:
!tree /content/gdrive/My\ Drive/darknet/

## Instalando as dependencias:

Checando a versão do CUDA

In [ ]:
# This cell can be commented once you checked the current CUDA version
# CUDA: Let's check that Nvidia CUDA is already pre-installed and which version is it. In some time from now maybe you
!/usr/local/cuda/bin/nvcc --version
!cat /etc/os-release


Instalando o cuDNN de versão compativel com a versão do CUDA (você deve baixar a versão correta e salva-la no drive), instale a versão correta [aqui](https://developer.nvidia.com/rdp/cudnn-archive).

In [ ]:
# Não esqueça de substituir o arquivo pelo nome correto que você o salvou no drive
!tar -xJvf gdrive/My\ Drive/darknet/cuDNN/cudnn-linux-x86_64-8.9.7.29_cuda12-archive.tar.xz -C /usr/local/
# !ls /usr/local/

Copiando o conteudo para o local correto:

In [ ]:
!cp /usr/local/cudnn-linux-x86_64-8.9.7.29_cuda12-archive/include/* /usr/local/cuda/include/
!cp /usr/local/cudnn-linux-x86_64-8.9.7.29_cuda12-archive/lib/* /usr/local/cuda/lib64/
# !ls /usr/local/cuda/include | grep cudnn

## Instalando a rede YOLOv3

Clonando o repositório com da darknet e salvando no drive [comente após a primeira execução].

Antes de executar essa célula tenha certeza que a pasta /darknet/bin foi criada no Drive.

In [ ]:
# Comment this code on the future runs.
!git clone https://github.com/kriyeng/darknet/
%cd /content/darknet

# Check the folder
!ls

#Compile Darknet
!make

#Copies the Darknet compiled version to Google drive
!cp ./darknet /content/gdrive/My\ Drive/darknet/bin/darknet

Função para mostrar imagens no notebook:

In [ ]:
def imShow(path):
  import cv2
  import matplotlib.pyplot as plt
  %matplotlib inline

  image = cv2.imread(path)
  height, width = image.shape[:2]
  resized_image = cv2.resize(image,(3*width, 3*height), interpolation = cv2.INTER_CUBIC)

  fig = plt.gcf()
  fig.set_size_inches(18, 10)
  plt.axis("off")
  #plt.rcParams['figure.figsize'] = [10, 5]
  plt.imshow(cv2.cvtColor(resized_image, cv2.COLOR_BGR2RGB))
  plt.show()

## Testando a rede com um exemplo

Obtendo os pesos pré treinados da rede YOLOv3:

In [ ]:
!wget https://pjreddie.com/media/files/yolov3.weights

In [ ]:
# Execute darknet using YOLOv3 model with pre-trained weights to detect objects on 'person.jpg'
!./darknet detect cfg/yolov3.cfg yolov3.weights data/person.jpg -dont-show

# Show the result using the helper imgShow()
imShow('predictions.jpg')

## Obtendo o dataset da COCO

Criando o diretorio para o dataset:

In [ ]:
!mkdir /content/dataset/
%cd /content/dataset/

Realizando o download do dataset:

In [ ]:
!wget http://images.cocodataset.org/zips/train2017.zip
!wget http://images.cocodataset.org/zips/val2017.zip
!wget http://images.cocodataset.org/annotations/annotations_trainval2017.zip

Descompactando os dados:

In [ ]:
!unzip train2017.zip
!unzip val2017.zip
!unzip annotations_trainval2017.zip

In [ ]:
# Lendo as classes
with open('/content/darknet/data/coco.names', 'r') as f:
    classes = [line.strip() for line in f.readlines()]

print("Classes do COCO:")
for i, class_name in enumerate(classes):
    print(f"{i}: {class_name}")

Criando o arquivo de configurações do dataset:

In [ ]:
%cd /content/darknet
!echo "classes= 80" > data/coco.data
!echo "train= data/train.txt" >> data/coco.data
!echo "valid= data/val.txt" >> data/coco.data
!echo "names= data/coco.names" >> data/coco.data
!echo "backup= backup/" >> data/coco.data

## Treinando o modelo

Alterando as configurações da rede para realizar o treinamento:

In [ ]:
%cd /content/darknet/cfg
!sed -i 's/batch=1/batch=64/' yolov3.cfg
!sed -i 's/subdivisions=1/subdivisions=16/' yolov3.cfg

!grep "batch=" yolov3.cfg
!grep "subdivisions=" yolov3.cfg

Criando o arquivo train.txt:

In [ ]:
import os

train_images_path = "/content/dataset/train2017"
train_txt_path = "/content/darknet/data/train.txt"

# Obter todos os arquivos de imagem no diretório
with open(train_txt_path, "w") as f:
    for image_file in os.listdir(train_images_path):
        if image_file.endswith(".jpg"):  # Verificar se é um arquivo de imagem
            f.write(f"{train_images_path}/{image_file}\n")

print(f"Arquivo train.txt criado em: {train_txt_path}")


In [ ]:
%cd /content/darknet/data/
!ls

Realizando o treinamento:

In [ ]:
%cd /content/darknet/
!./darknet detector train data/coco.data cfg/yolov3.cfg yolov3.weights -dont_show

## Testando o modelo treinado

Imagem de teste obtida de: https://oeco.org.br/noticias/26672-queremos-cidades-para-pessoas-e-cachorros/

In [ ]:
!./darknet detector test data/coco.data cfg/yolov3.cfg backup/yolov3_final.weights /content/teste.jpg
imShow('predictions.jpg')